# Credit Card Prediction Model in Redshift

Ref - https://aws.amazon.com/blogs/big-data/create-train-and-deploy-multi-layer-perceptron-mlp-models-using-amazon-redshift-ml/

Here we use the Credit Card Fraud detection data available at https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud  to create, train and deploy MLP model which
can be used further to identify fraudulent transactions from the newly captured transaction records.

For that, we have downloaded the dataset from the mentioned URL and identified the schema of the CSV file that comes with the downloaded content.

We first create a table in Amazon Redshift which should hold the data.
One can even keep this CSV file in S3, crawl it using AWS Glue and/or catalog it using Amazon Athena to prepare an external table which can be queried for training the MLP model.

Here we choose the option to create a table inside the Amazon Redshift cluster (or Amazon Redshift serverless endpoint).

In [ ]:
DROP TABLE IF EXISTS creditcardsfrauds;
CREATE TABLE creditcardsfrauds (
	txtime integer,
    v1 float8,
    v2 float8,
    v3 float8,
    v4 float8,
    v5 float8,
    v6 float8,
    v7 float8,
    v8 float8,
    v9 float8,
    v10 float8,
    v11 float8,
    v12 float8,
    v13 float8,
    v14 float8,
    v15 float8,
    v16 float8,
    v17 float8,
    v18 float8,
    v19 float8,
    v20 float8,
    v21 float8,
    v22 float8,
    v23 float8,
    v24 float8,
    v25 float8,
    v26 float8,
    v27 float8,
    v28 float8,
    amount float8,
    class integer
);

Now we load the data into this table

In [ ]:
COPY creditcardsfrauds
FROM 's3://redshift-ml-blog-mlp/creditcard.csv' 
IAM_ROLE default
CSV QUOTE as '\"' delimiter ',' IGNOREHEADER 1 maxerror 100
REGION 'us-east-1';

Now we do some quick checks on the numbers…

In [ ]:
select count(*) from creditcardsfrauds;

...and decide here the cutoff that will give you 70% or 80% of the total data to be considered for training. Remaining 30% or 20% should be used for validation

In [ ]:
CREATE model creditcardsfrauds_mlp
FROM (select * from creditcardsfrauds where txtime < 120954)
TARGET class 
FUNCTION creditcardsfrauds_mlp_fn
IAM_ROLE DEFAULT
MODEL_TYPE MLP
SETTINGS (
      S3_BUCKET '<<your-amazon-s3-bucket>>'',
      MAX_RUNTIME 54000
);

Keep checking the status of the model creation using the following command

In [ ]:
show model creditcardsfrauds_mlp;

Check the model explainability to understand which columns contributed effectively to derive the prediction

In [ ]:
SELECT json_table.report.explanations.kernel_shap.label0.global_shap_values 
FROM (select explain_model('creditcardsfrauds_mlp') as report) as json_table;

Optional section to check the counts. Can remove from the final post

In [ ]:
select min(txtime), max(txtime) from creditcardsfrauds limit 1;

select * from creditcardsfrauds limit 10;

select min(txtime), max(txtime), count(*) from creditcardsfrauds_mlp where time1 < 120954

select class, creditcards_mlp_fn(time1,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,amount) from creditcardsfrauds where time1 >= 120954 limit 50;

Validate the held off 20% to 30% data and check whether the predicted data is closer to the actual data and how much performance accuracy do we achieve.

In [ ]:
select actualvspredicted, count(actualvspredicted)
FROM (
	SELECT 
      CASE WHEN class =  
      creditcardsfrauds_mlp_fn(txtime,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,amount) 
      THEN 'PredictedMatchesActual' 
      else 'NoMatch' 
      END as actualvspredicted
    FROM creditcardsfrauds 
    WHERE txtime >= 120954
) 
group by actualvspredicted;

Now you can start predicting on the newly arriving data.

CREATE A STAGING TABLE TO HOLD NEWLY ARRIVING DATA FROM THE SOURCE WHICH WILL NOT CONAIN THE CLASS COLUMN - AS IT IS TO BE PREDICTED

In [ ]:
DROP TABLE if exists creditcardsfrauds_staging;
CREATE TABLE creditcardsfrauds_staging as (select * from creditcardsfrauds limit 0);
Alter table creditcardsfrauds_staging drop column class;

LETS CONSIDER ONLY ONE RECORD HERE WHICH HAS NEWLY ARRIVED

In [ ]:
insert into creditcardsfrauds_staging values(174965,-39999.11383160738512,0.58586417180689,-5.39973021073242,1.81709247345531,-0.840618465991056,-2.94354779071974,-2.20800192003372,1.05873267723056,-1.63233334974982,-5000.24598383776964,11.93351953683592,-53046479695456,-1.12745457501155,-666666.41662797597451,0.141237234328704,-2.54949823633632,-4.61471706851594,-10.47813794126038,-0.0354803664667244,0.306270740368093,0.583275998701341,-0.269208637986581,-0.456107772584008,-0.183659129549716,-0.328167759255761,0.606115810329683,0.884875539542905,-0.253700318894381,-2450000000);

USE THE FUNCTION TO PREDICT THE VALUE OF CLASS ALONG WITH THE PROBABILITY

In [ ]:
select predictedActive.labels[0], predictedActive.probabilities[0] 
from (
SELECT creditcardsfrauds_mlp_fn_prob(txtime,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,amount)as predictedACtive
FROM creditcardsfrauds_staging ) temp